In [1]:
import os

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [3]:
from LemonClassifier.constants import*
from LemonClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Importing Data

In [5]:
import os
import urllib.request as request
import zipfile
from LemonClassifier import logger
from LemonClassifier.utils.common import get_size


In [6]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-23 15:04:24,002] 20 LemonClassifier - INFO - yaml file: /Users/renyue/Desktop/Project/goodbad_lemon/config/config.yaml loaded successfully]
[2023-08-23 15:04:24,005] 20 LemonClassifier - INFO - yaml file: /Users/renyue/Desktop/Project/goodbad_lemon/params.yaml loaded successfully]
[2023-08-23 15:04:24,006] 34 LemonClassifier - INFO - created directory at: artifacts]
[2023-08-23 15:04:24,007] 34 LemonClassifier - INFO - created directory at: artifacts/data_ingestion]


HTTPError: HTTP Error 403: Forbidden